In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import operator
import gc
import time

import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import string
import re

import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

from keras.layers import add, Dense, Input, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras import backend as K
from keras import models
from keras.engine.topology import Layer
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
dev = pd.read_csv('../csv-data/BEST&MOST200/dev-best200.csv')
train = pd.read_csv('../csv-data/BEST&MOST200/train-best200.csv')
train = pd.concat([dev, train], axis=0)
test = pd.read_csv('../csv-data/BEST&MOST200/test-best200.csv')

In [3]:
X = train.drop(['tweet-id', 'user-id', 'class'], axis=1)
X_test = test.drop(['tweet-id', 'user-id', 'class'], axis=1)

# mmsc = MinMaxScaler()
# X = mmsc.fit_transform(X)
# X_test = mmsc.transform(X_test)

ss = StandardScaler()
X = ss.fit_transform(X)
X_test = ss.transform(X_test)

y = train['class']
encoder = LabelEncoder()
encoder.fit(y)
y_one_hot = encoder.transform(y)
y_one_hot = np_utils.to_categorical(y_one_hot)

In [4]:
folds = StratifiedKFold(n_splits=10, random_state=2019)
fold_num = 1

for train_idx, val_idx in folds.split(X, y):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_one_hot[train_idx], y_one_hot[val_idx]

    gc.collect()
    K.clear_session()

    model = Sequential()
    model.add(Dense(128, input_shape=(X.shape[1],)))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(64))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(32))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=5)
    mcp = ModelCheckpoint(filepath=f'./NN-models/models-without-feature-engineering/fold-{fold_num}.hdf5', 
                          monitor='val_acc', mode='max', 
                          verbose=1, 
                          save_best_only=True)
    
    model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              batch_size=32,
              epochs=50,
              verbose=1,
              callbacks=[mcp, es]
             )
    
    fold_num += 1






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 117550 samples, validate on 13063 samples
Epoch 1/50
117550/117550 [==============================] - 39s 333us/step - loss: 0.8843 - acc: 0.6359 - val_loss: 0.8375 - val_acc: 0.6548

Epoch 00001: val_acc improved from -inf to 0.65475, saving model to ./NN-models/models-without-feature-engineering/fold-1.hdf5
Epoch 2/50
117550/117550 [==============================] - 31s 265us/step - loss: 0.8476 - acc: 0.6518 - val_loss: 0.8389 - val_acc: 0.6547

Epoch 00002: val_acc did not improve from 0.65475
Epoch 3/50
117550/117550 [==============================] - 32s 268us/step - loss: 0.8417 - acc: 0.6537 - val_loss: 0.8401 - val_acc: 0.6515

Epoch 00003: val_acc did not improve from 0.65475
Epoch 4/50
117550/117550 [==============================] - 33s 280us/step - loss

117551/117551 [==============================] - 30s 254us/step - loss: 0.8349 - acc: 0.6568 - val_loss: 0.8548 - val_acc: 0.6488

Epoch 00006: val_acc improved from 0.64829 to 0.64883, saving model to ./NN-models/models-without-feature-engineering/fold-3.hdf5
Epoch 7/50
117551/117551 [==============================] - 30s 258us/step - loss: 0.8341 - acc: 0.6564 - val_loss: 0.8538 - val_acc: 0.6475

Epoch 00007: val_acc did not improve from 0.64883
Epoch 8/50
117551/117551 [==============================] - 30s 255us/step - loss: 0.8336 - acc: 0.6571 - val_loss: 0.8564 - val_acc: 0.6468

Epoch 00008: val_acc did not improve from 0.64883
Epoch 9/50
117551/117551 [==============================] - 30s 255us/step - loss: 0.8331 - acc: 0.6571 - val_loss: 0.8601 - val_acc: 0.6480

Epoch 00009: val_acc did not improve from 0.64883
Epoch 10/50
117551/117551 [==============================] - 30s 254us/step - loss: 0.8332 - acc: 0.6570 - val_loss: 0.8580 - val_acc: 0.6479

Epoch 00010: val_acc


Epoch 00006: val_acc did not improve from 0.65623
Epoch 7/50
117552/117552 [==============================] - 34s 293us/step - loss: 0.8355 - acc: 0.6565 - val_loss: 0.8458 - val_acc: 0.6539

Epoch 00007: val_acc did not improve from 0.65623
Epoch 8/50
117552/117552 [==============================] - 34s 289us/step - loss: 0.8352 - acc: 0.6561 - val_loss: 0.8442 - val_acc: 0.6571

Epoch 00008: val_acc improved from 0.65623 to 0.65707, saving model to ./NN-models/models-without-feature-engineering/fold-7.hdf5
Epoch 9/50
117552/117552 [==============================] - 33s 278us/step - loss: 0.8343 - acc: 0.6565 - val_loss: 0.8448 - val_acc: 0.6549

Epoch 00009: val_acc did not improve from 0.65707
Epoch 10/50
117552/117552 [==============================] - 32s 269us/step - loss: 0.8341 - acc: 0.6562 - val_loss: 0.8470 - val_acc: 0.6526

Epoch 00010: val_acc did not improve from 0.65707
Epoch 11/50
117552/117552 [==============================] - 31s 266us/step - loss: 0.8337 - acc: 0.

In [6]:
val_prob = np.zeros((len(train), 3), dtype=np.float32)
pred_prob = np.zeros((len(test), 3), dtype=np.float32)

folds = StratifiedKFold(n_splits=10, random_state=2019)
fold_num = 1

for train_idx, val_idx in folds.split(X, y):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_one_hot[train_idx], y_one_hot[val_idx]

    gc.collect()
    K.clear_session()
    
    model = models.load_model(f'./NN-models/models-without-feature-engineering/fold-{fold_num}.hdf5')
    val_prob[val_idx] = model.predict(X_val, batch_size=32, verbose=1)
    pred_prob = model.predict(X_test, batch_size=32, verbose=1) / 10
    fold_num += 1

32977/32977 [==============================] - 2s 47us/step


In [7]:
validation = train[['user-id', 'class']]
validation['prediction'] = encoder.inverse_transform(np.argmax(val_prob, axis=1))
accuracy_score(validation['class'], validation['prediction'])

0.655853552096652